In [ ]:
import numpy as np
def single_parity_check(bit_string,is_even): 
    if(len(bit_string)<=1):
        return "Number of bits should be atleast 2..!"
    print("Received Message:",bit_string)
    number_of_Ones=bit_string.count('1');
    if(is_even):
        if(number_of_Ones%2==0):
            return "Message accepted.No error detected..!"
        else:
            return "Message discard.Error detected..!"
    else:
        if(number_of_Ones%2==0):
            return "Message discard.Error detected..!"
        else:
            return "Message accepted.No error detected..!"
        
def message_with_2D_parity_check(bit_string,row,col):
    twoD_parity=np.array(list(bit_string),dtype=int)
    twoD_parity=twoD_parity.reshape(row,col)
    number_of_Ones=np.count_nonzero(twoD_parity,axis=1)  #number of ones row-wise
    number_of_Ones=number_of_Ones%2                      #set parity value
    twoD_parity=np.append(twoD_parity,number_of_Ones.reshape(row,-1),axis=1)  #append rows parity
    number_of_Ones=np.count_nonzero(twoD_parity,axis=0)    #number of ones column-wise 
    number_of_Ones=number_of_Ones%2                        #set parity values
    twoD_parity=np.append(twoD_parity,number_of_Ones.reshape(1,-1),axis=0)    #append column parity
    print("message is send as following train of bits moving towards left..\n")
    row,col=twoD_parity.shape
    for i in range(row):
        for j in range(col):
            print(twoD_parity[i,j],end="")
        print(end=" ")

        
def twoD_parity_check(twoD_parity,row,col):
    row_parity=np.count_nonzero(twoD_parity[0:row-1],axis=1)
    col_parity=np.count_nonzero(twoD_parity,axis=0)
    row_parity=row_parity%2==0
    col_parity=col_parity%2==0
    if(all(row_parity) and all(col_parity)):
        print("No Error Detected")
    elif(all(row_parity) or all(col_parity)):  #if error detected at rows or cols not both
        if(not all(row_parity)):
            print("Error Detected at Row Numbers:",end=" ")
            corrupted_row=np.where(row_parity==False) [0]
            corrupted_row=corrupted_row+1
            print(corrupted_row)
        else:
            print("Error Detected at Column Numbers:",end=" ")
            corrupted_col=np.where(col_parity==False) [0]
            corrupted_col=corrupted_col+1
            print(corrupted_col)                
    else:    
        corrupted_row=np.where(row_parity==False) [0]
        corrupted_col=np.where(col_parity==False) [0]
        #To start indexes from 1 increment by 1
        corrupted_row=corrupted_row+1
        corrupted_col=corrupted_col+1
        print("There may be error at:",end=" ")
        for i in range(len(corrupted_row)):
            for j in range(len(corrupted_col)):
                print(f"({corrupted_row[i]},{corrupted_col[j]})",end=" ")
    print("\nNOTE:If two bits in one data unit are corrupted and two bits\nexactly the same position in another data unit are also corrupted\nthen 2D Parity checker will not be able to detect the error.")
                
        
def message_with_hamming_algo(bit_string):
    n=len(bit_string)
    r=0
    while(n+r+1>pow(2,r)):  #calculate redundant bits
        r+=1
    li=list(bit_string)
    li.insert(0," ")   #to make calculation easy start bits from index 1  
    for i in range(r): #make parity bits room  
        li.insert(pow(2,i)," ")
    for i in range(r):
        li[pow(2,i)]=str(parity_bit_value_for_hamming(li,i+1))
        
    codeword="".join(li)
    print("Dataword: ",bit_string)
    print("Codeword: ",codeword)

def hamming_algo(bit_string):
    n=len(bit_string)
    i=0
    li=list(bit_string)  #convert to list to make operation easy
    li.insert(0," ")   #to make calculation easy start bits from index 1
    received_parity=[]
    while(pow(2,i)<n):              #retrieve parity bits
        received_parity.append(li[pow(2,i)])
        i+=1
    number_of_parity=len(received_parity)
    calculated_parity=[]
    for i in range(number_of_parity):   #calculate parity values
        calculated_parity.append(str(parity_bit_value_for_hamming(li,i+1)))
    calculated_parity=np.array(calculated_parity) 
    received_parity=np.array(received_parity)     #convert to np array to make calculation easy
    result=calculated_parity==received_parity     #determine which parities not verified
    print("Received Message:",bit_string)
    if(all(result)):
        print("No 1-bit error occur..!")
    else:
        bit_number=0
        for i in range(len(result)):
            if(result[i]==False):
                bit_number+=pow(2,i)    #determine corrupted bit
        print("Error detected and it is at bit number#",bit_number)
        if(li[bit_number]=='1'):
            li[bit_number]='0'
        else:
            li[bit_number]='1'
        print("Corrected Message:","".join(li))
        
    

def parity_bit_value_for_hamming(bit_string,parity_number):
    number_of_Ones=0
    j=1
    flag=False
    length=len(bit_string)
    while(not flag):        
        i=0
        while(i<pow(2,parity_number-1)):   #control how many consective bits consider for particular parity
            index=(pow(2,parity_number-1)*j)+i 
            
            if(index>=length):
                flag=True
                break
            if(index!=pow(2,parity_number-1)):
                if(bit_string[index]=='1'):
                    number_of_Ones+=1
            i+=1
        j+=2
    return number_of_Ones%2    
 
def make_multiple_of_chunksize(bit_string,chunksize):
    length=len(bit_string)      
    index=length-length%chunksize
    padding=(chunksize-length%chunksize)*"0"
    return bit_string[ :index]+padding+bit_string[index: ]

def make_checksum(bit_string,chunksize):
    checksum=0
    for i in range(0,len(bit_string),chunksize):
        checksum+=int(bit_string[i:i+chunksize],2)
    checksum=bin(checksum)[2:]
    if(len(checksum)>chunksize):        
        ind=len(checksum)%chunksize
        checksum=int(checksum[ind:],2)+int(checksum[0:ind],2)
        checksum=bin(checksum)[2:]
    if(len(checksum)<chunksize):
        checksum=(chunksize-len(checksum))*"0"+checksum
    li=list(checksum)   #convert to list to make assignment at particular index possible
    for i in range(len(li)):
        if(li[i]=='1'):
            li[i]='0'
        else:
            li[i]='1'
    checksum="".join(li)
    return checksum
    

def message_with_checksum(bit_string,chunksize):
    if(len(bit_string)%chunksize!=0):
        bit_string=make_multiple_of_chunksize(bit_string,chunksize)
    checksum=make_checksum(bit_string,chunksize)
    codeword=bit_string+checksum
    print("Dataword: ",bit_string)
    print("Codeword: ",codeword)
    
def checksum_algo(bit_string,chunksize):
    if(len(bit_string)%chunksize!=0):
        bit_string=(chunksize-len(bit_string)%chunksize)*"0"+bit_string
        print("\nLength of message is not multiple of chunk size.I do padding to make it compatible\n")
    print("Received Message:",bit_string)
    checksum=make_checksum(bit_string,chunksize)
    if(all(bit=='0' for bit in checksum)):
        print("No error Occur...!")
    else:
        print("Error detected..!Message discard")

def XOR(x,y):
    ans=[]
    for i in range(1,len(y)):
        if x[i]==y[i]:
            ans.append('0')
        else:
            ans.append('1')
    return "".join(ans)

def find_remainder(dividend,divisor):
    ind=len(divisor)
    length=ind
    div=dividend[ :ind]   #get portion of dividend to perform XOR
    len_of_dividend=len(dividend)
    while(ind<len_of_dividend):
        if div[0]=='1':
            div=XOR(div,divisor)+dividend[ind]  #perform XOR and then get one bit from upper
        else:                                   
            div=XOR(div,length*'0')+dividend[ind] #if leftmost 0 so need to get one more bit from upper
        ind+=1
        
    #need to perform last division to get answer
    
    if div[0]=='1':
        div=XOR(div,divisor)  #perform XOR and then get one bit from upper
    else:                                   
        div=XOR(div,length*'0') #if leftmost 0 so need to get one more bit from upper
    return div  

def message_with_CRC(bit_string,divisor):
    dividend=bit_string+(len(divisor)-1)*'0'
    rem=find_remainder(dividend,divisor)
    codeword=bit_string+rem
    print("Dataword: ",bit_string)
    print("Codeword: ",codeword)

def CRC_algo(bit_string,divisor):
    print("Received Message:",bit_string)
    rem=find_remainder(bit_string,divisor)
    if(all(bit=='0' for bit in rem)):
        print("No error Occur...!")
    else:
        print("Error detected..!Message discard")


#main


while(True):
    print("\nSelect what action you want to perform:")
    print("1.Detect One-bit error by parity bit\n2.View Codeword in case 2D parity check\n3.Detect multiple errors by 2D parity check")
    print("4.View Codeword in case Hamming Algorithm\n5.Detect and correct one-bit error by Hamming Algorithm\n6.View Codeword in case Cyclic Redundancy Check")
    print("7.Detect errors by Cyclic Redundancy Check\n8.View Codeword in case Checksum\n9.Detect errors by Checksum\n0.Exit\n")   
    choice = int(input("Enter your choice: "))
    while(choice<0 or choice>9):
        print("Invalid Choice! Try Again.")
        choice=int(input("Choice:"))
    if(choice==0):
        break
    elif(choice==1):
        print("Select option:\n1.Use Even Parity\n2.Use Odd Parity")
        parity_choice=int(input("Choice:"))
        while(parity_choice<1 or parity_choice>2):
            print("Invalid Choice! Try Again.")
            parity_choice=int(input("Choice:"))
        is_even=True
        if(parity_choice==2):
            is_even=False
        
        binary_string=input("Enter bit string:")
        while(not all(binary in '01' for binary in binary_string)):
            print("Invalid bit string.! Try again.")
            binary_string=input("Enter bit string:")
        
        print(single_parity_check(binary_string,is_even))
    
    elif(choice==2):
        
        binary_string=input("Enter bit string:")
        while(not all(binary in '01' for binary in binary_string)):
            print("Invalid bit string.! Try again.")
            binary_string=input("Enter bit string:")
        length=len(binary_string)    
        print("Enter number of rows and columns") 
        row=int(input("Number of Rows:"))
        col=int(input("Number of Columns:"))
        while(row*col<length):
            print("Invalid number of rows and column..!Try again")
            row=int(input("Number of Rows:"))
            col=int(input("Number of Columns:"))
        if(row*col>length):
            print("\nYour entered number of row and col not match length of your binary.I do padding with 0 to make it compatible\n")
            binary_string=(row*col-length)*'0'+binary_string
        
        message_with_2D_parity_check(binary_string,row,col)
        
    elif(choice==3):          
        print("Enter number of rows and columns (without including for parity bits)") 
        row=int(input("Number of Rows:"))
        col=int(input("Number of Columns:"))
        row=row+1    #add 1 row and col for parity bits
        col=col+1
        print("Enter Row-Wise")
        bit_str=input("Enter row:")
        while(len(bit_str)>col or  not all(binary in '01' for binary in bit_str)):
            print("Invalid row..! Try again.")
            bit_str=input("Enter row:")
        if(len(bit_str)<col):
            bit_str=(col-len(bit_str))*'0'+bit_str
        mat=np.array(list(bit_str),dtype=int).reshape(1,-1)
        for i in range(1,row):
            bit_str=input("Enter row:")
            while(len(bit_str)>col or not all(binary in '01' for binary in bit_str)):
                print("Invalid row..! Try again.")
                bit_str=input("Enter row:")
            if(len(bit_str)<col):
                bit_str=(col-len(bit_str))*'0'+bit_str
            mat=np.append(mat,np.array(list(bit_str),dtype=int).reshape(1,-1),axis=0)
        twoD_parity_check(mat,row,col)

    elif(choice==4):
        binary_string=input("Enter bit string:")
        while(not all(binary in '01' for binary in binary_string)):
            print("Invalid bit string.! Try again.")
            binary_string=input("Enter bit string:")
        message_with_hamming_algo(binary_string)
    elif(choice==5):
        binary_string=input("Enter bit string:")
        while(not all(binary in '01' for binary in binary_string)):
            print("Invalid bit string.! Try again.")
            binary_string=input("Enter bit string:")
        hamming_algo(binary_string)
    elif(choice==6):
        binary_string=input("Enter bit string:")
        while(not all(binary in '01' for binary in binary_string)):
            print("Invalid bit string.! Try again.")
            binary_string=input("Enter bit string:")
        
        divisor=input("Enter divisor bit string:")
        while(not all(binary in '01' for binary in divisor) or len(divisor)>len(binary_string)):
            print("Invalid divisor bit string.! Try again.")
            divisor=input("Enter divisor bit string:")
        message_with_CRC(binary_string,divisor)
    elif(choice==7):
        binary_string=input("Enter bit string:")
        while(not all(binary in '01' for binary in binary_string)):
            print("Invalid bit string.! Try again.")
            binary_string=input("Enter bit string:")
        
        divisor=input("Enter divisor bit string:")
        while(not all(binary in '01' for binary in divisor) or len(divisor)>len(binary_string)):
            print("Invalid divisor bit string.! Try again.")
            divisor=input("Enter divisor bit string:")
        CRC_algo(binary_string,divisor)
    elif(choice==8):
        binary_string=input("Enter bit string:")
        while(not all(binary in '01' for binary in binary_string)):
            print("Invalid bit string.! Try again.")
            binary_string=input("Enter bit string:")
        message_with_checksum(binary_string,8)
    elif(choice==9):
        binary_string=input("Enter bit string:")
        while(not all(binary in '01' for binary in binary_string)):
            print("Invalid bit string.! Try again.")
            binary_string=input("Enter bit string:")
        if(len(binary_string)<16):
            binary_string=(len(binary_string)-16)*'0'+binary_string
        
        checksum_algo(binary_string,8)
    
#message_with_checksum("10011001111000100010010010000100",8)   
#CRC_algo("10011101100","1001")
#message_with_CRC("10011101","1001")            


Select what action you want to perform:
1.Detect One-bit error by parity bit
2.View Codeword in case 2D parity check
3.Detect multiple errors by 2D parity check
4.View Codeword in case Hamming Algorithm
5.Detect and correct one-bit error by Hamming Algorithm
6.View Codeword in case Cyclic Redundancy Check
7.Detect errors by Cyclic Redundancy Check
8.View Codeword in case Checksum
9.Detect errors by Checksum
0.Exit

Enter your choice: 8
Enter bit string:11001011010100
Dataword:  1100101100010100
Codeword:  110010110001010000100000

Select what action you want to perform:
1.Detect One-bit error by parity bit
2.View Codeword in case 2D parity check
3.Detect multiple errors by 2D parity check
4.View Codeword in case Hamming Algorithm
5.Detect and correct one-bit error by Hamming Algorithm
6.View Codeword in case Cyclic Redundancy Check
7.Detect errors by Cyclic Redundancy Check
8.View Codeword in case Checksum
9.Detect errors by Checksum
0.Exit

Enter your choice: 8
Enter bit string:00110